![Py4Eng](img/logo.png)

# Calling C functions from Python with `ctypes`
## Yoav Ram


[ctypes](https://docs.python.org/3/library/ctypes.html) is a foreign function library for Python. It provides C compatible data types, and allows calling functions in DLLs or shared libraries. It can be used to wrap these libraries in pure Python.

In [1]:
import ctypes

We'll start by loading the standard C library (`msvcrt` on Windows, `libc` on Linux):

In [2]:
import sys
print(sys.platform)
if sys.platform == 'win32': # windows
    libc = ctypes.cdll.msvcrt
elif sys.platform == 'darwin': # osx
    libc = ctypes.CDLL('/usr/lib/libc.dylib')
else: # linux
    libc = ctypes.CDLL("libc.so.6")

darwin


In general, we can find the shared library path using `ctypes.util.find_library("c")`.

In [3]:
ctypes.util.find_library("c")

'/usr/lib/libc.dylib'

We can now get functions from the standard library,

In [4]:
libc.printf(b"Hello, %s\n", b"World!")
libc.printf(b"%d bottles of beer\n", 42)

19

`printf` prints to the real standard output channel, not to `sys.stdout`; look for the output in the notebook server console. 

> `None`, integers, bytes objects and (unicode) strings are the only native Python objects that can directly be used as parameters in these function calls. `None` is passed as a C NULL pointer, bytes objects and strings are passed as pointer to the memory block that contains their data (char * or wchar_t *). Python integers are passed as the platforms default C int type, their value is masked to fit into the C type.

However, `printf` expects ANSI and not unicode, so we need to pass it `bytes` rather than `str`:

In [5]:
libc.printf("Hello, %s\n", "World!")

1

Another example is the `time` function:

In [6]:
print(libc.time)
libc.time(None)

1740985861

Compare with Python's `time.time`:

In [7]:
import time
time.time()

1740985861.376348

So, Python's `time` gave us extra precision.

Here is another example, it uses the [`strchr`](http://www.cplusplus.com/reference/cstring/strchr/) function, which expects a string pointer and a char, and returns a pointer to a string that starts with that char:

In [8]:
libc.strchr(b"abcdef", ord("d")) 

315753395

By default functions are assumed to return the C `int` type. Other return types can be specified by setting the `restype` attribute of the function object. Since `strchr` returns a string, we need to set the result type to `c_char_p` - 

In [9]:
libc.strchr.restype = ctypes.c_char_p
libc.strchr(b"abcdef", ord("d")) 

b'def'

Similarly, functions expect integers, bytes, strings and `None`, so if we want to avoid the `ord("x")` call we can set the `argtypes` attribute, and the second argument will be converted from a single character Python object into a C wchar:

In [10]:
libc.strchr.argtypes = [ctypes.c_char_p, ctypes.c_wchar]
libc.strchr(b"abcdef", "d")

b'def'

Other ctypes types can be found in the [documentation](https://docs.python.org/3/library/ctypes.html#fundamental-data-types).

## Using your own C extension

We'll use a C extension that implements the Fibonacci series using recursion.

This example follows the [SciPy Cookbook](https://scipy-cookbook.readthedocs.org/items/Ctypes.html#fibonacci-example-using-numpy-arrays-c-and-scons).

First, let's change to the example directory:

In [11]:
%cd ../scripts/ctypes/

/Users/yoavram/Work/Teaching/Py4Eng/scripts/ctypes


Have a look at the C file. Note that we export the three functions (`fib`, `fibseries`, and `fibmatrix`) with `dllexport`, and that we don't use an `h`.

In [12]:
%less fibonacci.c

/*
    Filename: fibonacci.c
    To be used with fibonacci.py, as an imported library. Use Scons to compile,
    simply type 'scons' in the same directory as this file (see www.scons.org).
*/
#if defined(_MSC_VER)
    #define EXPORT __declspec(dllexport)
#else
    //  do nothing and hope for the best?
    #define EXPORT
#endif

EXPORT int fib(int);
EXPORT void fibseries(int *, int, int *);
EXPORT void fibmatrix(int *, int, int, int *);

/* Function prototypes */
int fib(int a);
void fibseries(int *a, int elements, int *series);
void fibmatrix(int *a, int rows, int columns, int *matrix);


int fib(int a)
{
    if (a <= 0) /*  Error -- wrong input will return -1. */
        return -1;
    else if (a==1)
        return 0;
    else if ((a==2)||(a==3))
        return 1;
    else
        return fib(a - 2) + fib(a - 1);
}

void fibseries(int *a, int elements, int *series)
{
    int i;
    for (i=0; i < elements; i++)
    {
    series[i] = fib(a[i]);
    }
}

void fibmatrix(int *a, int rows, i

There's nothing special here, and the same code could be written in Python, too, with some changes:

In [13]:
import numpy as np

def fib(a):
    if a <= 0: #  Error -- wrong input will return -1. 
        return -1
    elif a == 1:
        return 0;
    elif a == 2 or a == 3:
        return 1
    else:
        return fib(a - 2) + fib(a - 1)

def fibseries(a):
    series = np.empty_like(a, dtype=int)
    for i in range(len(a)):
        series[i] = fib(a[i])
    return series

def fibmatrix(a):
    rows, columns = a.shape
    matrix = np.empty_like(a, dtype=int)
    for i in range(rows):
        for j in range(columns):
            matrix[i,j] = fib(a[i,j])
    return matrix

We need to compile `fibonacci.c`. 
Choose the correct statement according to your operating system. 

Notes:
- on Windows, make sure `vcvarsall.bat` is in the `PATH` environment variable.
- if the compilation statement fails, try to run it from a terminal window instead of directly from the notebook.

In [ ]:
##  windows
# !cvarsall.bat amd64 & cl -LD fibonacci.c -Fefibonacci.dll
## mac 
!clang -dynamiclib -o libfibonacci.dylib fibonacci.c 
## linux 
#!clang -shared -o libfibonacci.so fibonacci.c 

Now that we have our shared library, let's have a look at the Python wrapper around `fibonacci.dll` (Windows), `libfibonacci.dylib` (MacOS), or `libfibonacci.so` (Linux):

In [18]:
%less fibonacci.py


"""
Filename: fibonacci.py
Demonstrates the use of ctypes with three functions:

    (1) fib(a)
    (2) fibseries(b)
    (3) fibmatrix(c)

See: https://scipy-cookbook.readthedocs.org/items/Ctypes.html#fibonacci-example-using-numpy-arrays-c-and-scons
     http://www.lejordet.com/2009/04/simple-python-ctypes-example-in-windows/
"""
import numpy as np
import ctypes as ct
import sys
if sys.platform == 'win32': # win
    lib_filename = 'fibonacci.dll'
elif sys.platform == 'darwin': # osx
    lib_filename = 'libfibonacci.dylib'
else: # linux
    lib_filename = 'libfibonacci.so'
# Load the library as _libfibonacci.
# Why the underscore (_) in front of _libfibonacci below?
# To minimise namespace pollution -- see PEP 8 (www.python.org).
_libfibonacci = np.ctypeslib.load_library(lib_filename, '.')

_libfibonacci.fib.argtypes = [ct.c_int] #  Declare arg type, same below.
_libfibonacci.fib.restype  =  ct.c_int  #  Declare result type, same below.

_libfibonacci.fibseries.argtypes = [
    np.ctyp

This Python module loads the DLL, reads the relevant functions, annotates the argument and return value types, and then defines wrapper Python function that create the output arguments and communicate with the ctypes functions. There is also a lot of add docstrings. Note that we also make use of `numpy.ctypeslib`, which allows to use NumPy arrays wherever data buffers are used in the C code.

In [25]:
%ls

SConstruct         build.bat          fibonacci.py
__pycache__/       fibonacci.c        test_fibonacci.py


In [26]:
import fibonacci as fb

In [27]:
fb.fibseries([1, 5, 9])

array([ 0,  3, 21], dtype=int32)

Compare to the pure-Python function:

In [28]:
fibseries([1, 5, 9])

array([ 0,  3, 21])

In [29]:
%timeit fib(20)
%timeit fb.fib(20)

678 μs ± 1.11 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
20.9 μs ± 42.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [30]:
v = np.arange(1, 15)
%timeit fibseries(v)
%timeit fb.fibseries(v)

304 μs ± 5.94 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
11.8 μs ± 366 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [31]:
v = np.random.randint(5, 15, (5, 5))
%timeit fibmatrix(v)
%timeit fb.fibmatrix(v)

1.1 ms ± 5.69 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
19.9 μs ± 76.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


It's clear that the **C implementation is significantly faster than the pure Python implementation**, especially when given an array of inputs, despite the fast that it perfoms the exact same algorithm as the pure Python version does.

# References

- [ctypes](https://docs.python.org/3/library/ctypes.html)
- [SciPy Cookbook](https://scipy-cookbook.readthedocs.org/items/Ctypes.html)
- [IPython Cookbook](https://ipython-books.github.io/54-wrapping-a-c-library-in-python-with-ctypes/)'s chapter 5 has a section about using ctypes.

## Colophon
This notebook was written by [Yoav Ram](http://python.yoavram.com).
This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)